<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Labo-App_Mobile/IdEcorces-ResConv-TFLiteModelMaker-Colab.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

#  Identification d'arbres à partir de leur écorce

## Création et entraînement d'un modèle pour application mobile

### Réseau convolutif avec apprentissage par transfert et amplification des données

### Utilisation de `TensorFlow Lite Model Maker` avec `TensorFlow 2.0`

### Inspiration et droits d'auteur

Ce laboratoire s'inspire de plusieurs oeuvres en logiciels libres qui ont été transformées dont:

<a href="https://www.tensorflow.org/lite/guide/model_maker" target='_blank'>TensorFlow Lite Model Maker</a> - site Google / Guide TensorFlow pour mobile et objet connecté 

##### Copyright (c) 2017, François Chollet  
##### Copyright (c) 2019-2022, The TensorFlow Authors.
##### Copyright (c) 2022, Claude Coulombe

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

#### Données

Les données sur les écorces d'arbres proviennent de <a href="https://data.mendeley.com/research-data/?search=barknet">BarkNet</a>, une banque en données ouvertes sous licence MIT de 23 000 photos d'écorces d'arbres en haute résolution prises avec des téléphones intelligents par une équipe d'étudiants et de chercheurs du <a href="https://www.sbf.ulaval.ca/" target='_blank'>Département des sciences du bois et de la forêt de l'Université Laval</a> à Québec.</p>

## Chargement des bibliothèques Python

La bibliothèque `TensorFlow Lite Model Maker` simplifie le processus d'entraînement d'un modèle TensorFlow pour une solution embarquée comme un appareil mobile ou un objet connecté. Pour cela, `TensorFlow Lite Model Maker` utilise l'apprentissage par transfert avec un réseau convolutif de type `EfficientNet` pour réduire la quantité de données d'entraînement requises, raccourcir le temps d'entraînement et produire un modèle de faible taille.

In [ ]:
from tflite_model_maker import image_classifier
from tflite_model_maker.image_classifier import DataLoader
import tensorflow as tf
assert tf.__version__.startswith('2')

import matplotlib.pyplot as plt
import numpy as np

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>

**Note** : Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 42

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
tf.random.set_seed(GERME_ALEATOIRE)

# Note: Retrait du comportement déterministe
# à cause de keras.layers.RandomContrast(...)
# dont il n'existe pas de version déterministe
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Acquisition des données

In [2]:
dict_arbres = {
    'BOJ' : "Betula alleghaniensis - Bouleau jaune - Yellow birch",
    'BOP' : "Betula papyrifera - Bouleau à papier - White birch",
    'CHR' : "Quercus rubra - Chêne rouge - Northern red oak",
    'EPB' : "Picea glauca - Épinette blanche - White spruce",
    'EPN' : " Picea mariana - Épinette noire - Black spruce",
    'EPO' : "Picea abies - Épinette de Norvège - Norway spruce",
    'EPR' : "Picea rubens - Épinette rouge - Red spruce",
    'ERB' : "Acer platanoides - Érable de Norvège - Norway maple",
    'ERR' : "Acer rubrum - Érable rouge - Red maple",
    'ERS' : "Acer saccharum - Érable à sucre - Sugar maple",
    'FRA' : "Fraxinus americana - Frêne d'Amérique - White ash",
    'HEG' : "Fagus grandifolia - Hêtre à grandes feuilles - American beech",
    'MEL' : "Larix laricina - Mélèze - Tamarack",
    'ORA' : "Ulmus americana - Orme d'Amérique - American elm",
    'OSV' : "Ostrya virginiana - Ostryer de Virginie - American hophornbeam",
    'PEG' : "Populus grandidentata - Peuplier à grandes dents - Big-tooth aspen",
    'PET' : "Populus tremuloides - Peuplier faux tremble - Quaking aspen",
    'PIB' : "Pinus strobus - Pin blanc - Eastern white pine",
    'PID' : "Pinus rigida - Pin rigide - Pitch pine",
    'PIR' : "Pinus resinosa - Pin rouge - Red pine",
    'PRU' : "Tsuga canadensis - Pruche du Canada - Eastern Hemlock",
    'SAB' : "Abies balsamea - Sapin Baumier - Balsam fir",
    'THO' : "Thuja occidentalis - Thuya occidental - Northern white cedar",
}

print("Dictionnaire mémorisé")

Dictionnaire mémorisé


### Création des répertoires de données

Nous allons créer un répertoire de base `donnees`, un répertoire `lab_ecorces` où les données seront réparties en données d'entraînement, de validation et de test pour chaque classe cible.

Enfin, un répertoire `modeles` pour mémoriser les modèles une fois entraînés.

In [ ]:
try:
    os.mkdir("/content/donnees/")
except OSError:
    pass
try:
    os.mkdir("/content/modeles/")
except OSError:
    pass

### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>


8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données réduit « barknet-mini » de 1.5 Go avec la commande suivante:

In [ ]:
# Attention! Jeu réduit de données 1.5 Go - plus rapide à télécharger et à traiter
!kaggle datasets download claudecoulombe/barknet-mini --unzip -p /content/donnees/
repertoire_donnees = "/content/donnees/BarkNet-mini"

#### Retrait des données sur l'érable de Norvège, le Peuplier à grandes dents et le Pin rigide

Il n'y a qu'un seul spécimen d'érable de Norvège (Acer platanoides), 3 spécimens de Peuplier à grandes dents (Populus grandidentata) et 4 spécimens de Pin rigide (Pinus rigida). La solution simple est de supprimer ces trois classes.

In [ ]:
!rm -r "/content/donnees/BarkNet/ERB"
!rm -r "/content/donnees/BarkNet/PEG"
!rm -r "/content/donnees/BarkNet/PIR"

### Lecture / chargement des données

In [ ]:
donnees = DataLoader.from_folder(repertoire_donnees)

### Découpage des données entre données d'entraînement et données de test

In [ ]:
donnees_entrainement, donnees_test = donnees.split(0.9)

## Création & entraînement d'un modèle d'apprentissage par transfert

In [ ]:
modele_de_transfert = image_classifier.create(donnees_entrainement,
                                              use_augmentation=True,
                                              epochs=50,
                                              dropout_rate=0.2)

### Évaluation du modèle avec les données de test

In [ ]:
erreur, exactitude = modele_de_transfert.evaluate(donnees_test)
print("Exactitude données de test:   %0.2f" % exactitude)

In [ ]:
liste_noms_classes = donnees_entrainement.index_to_label
liste_noms_classes

In [ ]:
predictions = modele_de_transfert.predict_top_k(donnees_test)
etiquettes_predites = [resultat[0][0] for resultat in predictions]
print(etiquettes_predites)

In [ ]:
len(etiquettes_predites)

In [ ]:
def obtenir_classes_du_tfdataset(tfdataset, batched=False):
    etiquettes = list(map(lambda x: x[1], tfdataset)) 
    if not batched:
        return tf.concat(etiquettes, axis=0)
    return etiquettes

In [ ]:
listes_numeros_arbres = obtenir_classes_du_tfdataset(donnees_test)._numpy().tolist()
print(listes_numeros_arbres)
vraies_etiquettes = [liste_noms_de_classes[no_arbres] for no_arbres in listes_numeros_arbres]
print(vraies_etiquettes)


In [ ]:
from sklearn import metrics
score = metrics.accuracy_score(vraies_etiquettes, etiquettes_predites)
print("Exactitude:   %0.2f" % score)


### Matrice de confusion

In [ ]:
# https://stackoverflow.com/questions/65618137/confusion-matrix-for-multiple-classes-in-python

import itertools

def afficher_matrice_de_confusion(matrice_confusion_brute, classes,
                          normalisation=False,
                          titre='Matrice de confusion',
                          carte_des_couleurs=plt.cm.Blues):
    plt.figure(figsize=(14,12))
    plt.imshow(matrice_confusion_brute, interpolation='nearest', cmap=carte_des_couleurs)
    plt.title(titre)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalisation:
        matrice_confusion_brute = matrice_confusion_brute.astype('float') / matrice_confusion_brute.sum(axis=1)[:, np.newaxis]
        print("Matrice de confusion normalisée")
    else:
        print('Matrice de confusion non normalisée')

    seuil = matrice_confusion_brute.max() / 2.
    for i, j in itertools.product(range(matrice_confusion_brute.shape[0]), range(matrice_confusion_brute.shape[1])):
        plt.text(j, i, matrice_confusion_brute[i, j],
                 horizontalalignment="center",
                 color="white" if matrice_confusion_brute[i, j] > seuil else "black")

    plt.tight_layout()
    plt.ylabel('Vraies étiquettes')
    plt.xlabel('Étiquettes prédites')

print("Code affichage matrice de confusion")

In [ ]:
cm = metrics.confusion_matrix(vraies_etiquettes_index, predictions_index)
class_names = list(train_data.class_names)
plot_confusion_matrix(cm, classes=class_names)

matrice_confusion_brute = metrics.confusion_matrix(vraies_etiquettes,etiquettes_predites)
afficher_matrice_de_confusion(matrice_confusion_brute, classes=liste_noms_classes)


### Rapport de classification

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(vraies_etiquettes, etiquettes_predites, target_names=liste_noms_classes))

## Compresser et exporter le modèle en format `TensorFlow Lite`

In [ ]:
repertoire_modeles = "/content/modeles/" 
model.export(export_dir=repertoire_modeles+"ModeleEcorcIA.tflite")


## Télécharger le modèle en format `TensorFlow Lite`

In [ ]:
from google.colab import files
files.download(repertoire_modeles+"ModeleEcorceIA.tflite") 


In [ ]:
print("Fin de l'exécution du carnet IPython")